# Tensorflow model for MNIST x Solidity

Step 1: Train the model  
Step 2: Verify we export the model correctly  
Step 3: Export it for Solidity  
Step 4: Export it, but only some part  
Step 5: Compute accuracy of exported model: 60% 🚀😂😂😂🚀  

# Step 1: Train the model

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  #tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

# Step 2: Verify we export the model correctly

In [ ]:
n = 10
test = None
for a in ds_test:
  test = a[0][n]
  print(model.predict(a[0])[n])
  break

w = model.layers[1].get_weights()[0]
b = model.layers[1].get_weights()[1]

N = 28
res = [0]*10
for i in range(N):
  for j in range(N):
    for k in range(10):
      res[k] += test[i][j] * w[i*28+j][k]
for k in range(10):
  res[k] += b[k]
for e in res:
  print(int(float(e)), int((float(e) - int(float(e))) * 2**15))
for e in res:
  print(float(e), end=" ")

# Step3: Export it for Solidity

In [ ]:
def to_fixed(f):
  f = float(f)
  top = int(f)
  bot = int((f - top) * 2**15)
  return "NeuralNet.Point("+str(top)+","+str(bot)+")"

def print_tab(tab):
  s = ["["]
  for i in range(len(tab)):
    s.append("[")
    s.append(",".join([to_fixed(a) for a in tab[i]]))
    s.append("],")
  s = s[:-1]
  s.append("]]") # ?
  print("".join(s))

print_tab(test)
print_tab(w)
print(",".join(map(to_fixed, b)))

# Step3: Export to solidity, 2nd try



In [ ]:
def print_t(t, maxi, w, nom):
  l = []
  c = 0
  for i in range(len(w)):
    for j in range(len(w[i])):
      if abs(w[i][j]) > t:
        l.append((i, j))
        print(nom + "["+str(i)+"]["+str(j)+"] = " + to_fixed(w[i][j])+";")
        c += 1
        if c > maxi:
          print("OVERFLOW")
          return l
  print("#", c)
  return l


l = print_t(0.4, float("inf"), w, "w")
#print_t(0, 1000, test, "test")


# Compute and makes sure our solidity code is still accurate
res = [0]*10
for i in range(N):
  for j in range(N):
    for k in range(10):
      if (i*28+j, k) in l:
        res[k] += test[i][j] * w[i*28+j][k]
for k in range(10):
  res[k] += b[k]
for e in res:
  print(int(float(e)), int((float(e) - int(float(e))) * 2**15))
for e in res:
  print(float(e), end=" ")

# Rewrite the model and re-evaluate it

In [ ]:
for i in range(28*28):
  tmp = [w[i][j] * ((i, j) in l) for j in range(10)]
  model.layers[1].weights[0][i].assign(tf.constant(tmp))

print(model.evaluate(ds_test))